<a href="https://colab.research.google.com/github/Dangandy/reviews/blob/main/Prep1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the Data Analyst test!

**Congrats! You've made it this far!**

Here, we will test you on your SQL, Python, and some business / stats understanding. 
Please read each question carefully. We've left instructions and some additional notes where required.

**GL;HF**

NOTE: this was completed for someone to cross reference

## SQL

This section will test you on your SQL skills. You will be required to write SQL in Jupyter via the pandasql package -- if this is the first time you've done this - this [stackoverflow](https://stackoverflow.com/questions/45865608/executing-an-sql-query-over-a-pandas-dataset/45866311) link should help get you started.

*As a note: pandasql uses SQLite syntax*

TIP: google `sqlite "topic"`

#### These are your necessary imports to get you started

In [2]:
!pip install -U pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=6120150da4c1f00209da02cc0489f71ad46fee082c86d792b64543aa90636dc7
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [3]:
import pandas as pd
import numpy as np
# import pandasql as ps
from pandasql import sqldf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Q1 

Using the `payment_data_interview.csv` dataset, write the SQL query you’d use to calculate the following, and explain your steps:

1. Mean price point, by platform 
2. Median price point, by platform 
3. Mode price point, by platform


**Note: Do not use any pre-built median or mode functions.**

This should be the output of each table

``` 
|platform
|Mean, Median, or Mode (depending on question)
```


In [5]:
file_name = "/content/drive/MyDrive/test/uken_interview_data_analyst/payments_data_interview.csv"
payment_data_interview_df = pd.read_csv(file_name) ; payment_data_interview_df.head()

,user_id,event_date,platform,usd_price
0,16752,2020-04-01,android,1.99
1,18809,2020-04-01,ios,9.99
2,43576,2020-04-01,ios,4.99
3,67895,2020-04-01,android,29.99
4,59801,2020-04-01,android,4.99


#### Mean

In [6]:
"""
1. group by platform
2. calculate mean, medium, mode, trival...
"""
q1 = """SELECT platform, sum(usd_price)/count(platform) FROM payment_data_interview_df GROUP BY platform"""
pysqldf = lambda q: sqldf(q, globals())
pysqldf(q1)


,platform,sum(usd_price)/count(platform)
0,android,12.390000
1,ios,15.323333


In [7]:
# cross checking
q2 = """SELECT platform, avg(usd_price) FROM payment_data_interview_df GROUP BY platform"""
pysqldf(q2)

,platform,avg(usd_price)
0,android,12.390000
1,ios,15.323333


#### Mode

In [8]:
# The "mode" is the value that occurs most often.
# 1. count usd_price
# 2. get max count
q3 = """
Select platform, usd_price, max(t.Count) from
(Select platform, count(usd_price) as Count, usd_price FROM payment_data_interview_df GROUP BY usd_price) as t
group by platform
"""
pysqldf(q3)

,platform,usd_price,max(t.Count)
0,android,4.99,3
1,ios,1.99,3


#### Median
SOURCE: https://stackoverflow.com/questions/15763965/how-can-i-calculate-the-median-of-values-in-sqlite

In [9]:
# 0. see data
q0 = """
select * from payment_data_interview_df
"""
print(f"all: {pysqldf(q0)} \n")

# 1. get middle count for offset
q1 = """
SELECT
    (COUNT(*) - 1) / 2
FROM payment_data_interview_df
"""
print(f"middle offset: {pysqldf(q1)} \n")

# 2. check if data is even or odd
q2 = """
SELECT COUNT(*) % 2 FROM payment_data_interview_df
"""
print(f"even or odd? {pysqldf(q2)}\n")

# 3. select middle element of sorted df
q3 = """
SELECT
    platform,
    usd_price
FROM payment_data_interview_df
WHERE platform = 'ios'
ORDER BY usd_price
LIMIT 2 - (
    SELECT COUNT(*)
    FROM payment_data_interview_df
    WHERE platform = 'ios'
) % 2    -- odd 1, even 2
OFFSET (
    SELECT
        (COUNT(*) - 1) / 2
    FROM payment_data_interview_df
    WHERE platform = 'ios'
)
"""
print(f"middle element: {pysqldf(q3)}\n")

# 4. get average if even
q4 = """
select
    sum(usd_price)/count(usd_price) 
FROM (
    SELECT
        platform,
        usd_price
    FROM payment_data_interview_df
    WHERE platform = 'ios'
    ORDER BY usd_price
    LIMIT 2 - (
        SELECT COUNT(*)
        FROM payment_data_interview_df
        WHERE platform = 'ios'
    ) % 2    -- odd 1, even 2
    OFFSET (
        SELECT
            (COUNT(*) - 1) / 2
        FROM payment_data_interview_df
        WHERE platform = 'ios'
    )
)
"""
print(f"combine for one: {pysqldf(q4)}\n")

# 5 final..
q5 = """
SELECT
    platform,
    sum(usd_price)/count(usd_price) as 'median'
FROM (
    SELECT
        platform,
        usd_price
    FROM payment_data_interview_df
    WHERE platform = 'ios'
    ORDER BY usd_price
    LIMIT 2 - (
        SELECT COUNT(*)
        FROM payment_data_interview_df
        WHERE platform = 'ios'
    ) % 2    -- odd 1, even 2
    OFFSET (
        SELECT
            (COUNT(*) - 1) / 2
        FROM payment_data_interview_df
        WHERE platform = 'ios'
    )
)
UNION SELECT
    platform,
    sum(usd_price)/count(usd_price) 
FROM (
    SELECT
        platform,
        usd_price
    FROM payment_data_interview_df
    WHERE platform = 'android'
    ORDER BY usd_price
    LIMIT 2 - (
        SELECT COUNT(*)
        FROM payment_data_interview_df
        WHERE platform = 'android'
    ) % 2    -- odd 1, even 2
    OFFSET (
        SELECT
            (COUNT(*) - 1) / 2
        FROM payment_data_interview_df
        WHERE platform = 'android'
    )
)
"""
pysqldf(q5)


all:     user_id  event_date platform  usd_price
0     16752  2020-04-01  android       1.99
1     18809  2020-04-01      ios       9.99
2     43576  2020-04-01      ios       4.99
3     67895  2020-04-01  android      29.99
4     59801  2020-04-01  android       4.99
5     23456  2020-04-01      ios       2.99
6     78345  2020-04-01  android      19.99
7     71236  2020-04-01      ios      69.99
8     89234  2020-04-01      ios       1.99
9     43459  2020-04-01      ios       1.99
10    23598  2020-04-01  android       4.99 

middle offset:    (COUNT(*) - 1) / 2
0                   5 

even or odd?    COUNT(*) % 2
0             1

middle element:   platform  usd_price
0      ios       2.99
1      ios       4.99

combine for one:    sum(usd_price)/count(usd_price)
0                             3.99



,platform,median
0,android,4.99
1,ios,3.99


#### Combine all together

In [10]:
q1 = """
SELECT
    t1.platform,
    sum(t1.usd_price)/count(t1.platform) as 'mean',
    t2.mode,
    t3.median
FROM
    payment_data_interview_df as t1
LEFT JOIN (
    SELECT
        platform,
        usd_price as 'mode',
        max(Count)
    FROM (Select platform, count(usd_price) as Count, usd_price FROM payment_data_interview_df GROUP BY usd_price)
    group by platform
) AS t2
ON t1.platform = t2.platform
LEFT JOIN (
    SELECT
        platform,
        sum(usd_price)/count(usd_price) as 'median'
    FROM (
        SELECT
            platform,
            usd_price
        FROM payment_data_interview_df
        WHERE platform = 'ios'
        ORDER BY usd_price
        LIMIT 2 - (
            SELECT COUNT(*)
            FROM payment_data_interview_df
            WHERE platform = 'ios'
        ) % 2    -- odd 1, even 2
        OFFSET (
            SELECT
                (COUNT(*) - 1) / 2
            FROM payment_data_interview_df
            WHERE platform = 'ios'
        )
    )
    UNION SELECT
        platform,
        sum(usd_price)/count(usd_price) 
    FROM (
        SELECT
            platform,
            usd_price
        FROM payment_data_interview_df
        WHERE platform = 'android'
        ORDER BY usd_price
        LIMIT 2 - (
            SELECT COUNT(*)
            FROM payment_data_interview_df
            WHERE platform = 'android'
        ) % 2    -- odd 1, even 2
        OFFSET (
            SELECT
                (COUNT(*) - 1) / 2
            FROM payment_data_interview_df
            WHERE platform = 'android'
        )
    )
) as t3
ON t1.platform = t3.platform
GROUP BY t1.platform
"""
pysqldf(q1)

,platform,mean,mode,median
0,android,12.390000,4.99,4.99
1,ios,15.323333,1.99,3.99


In [11]:
pysqldf('SELECT * FROM payment_data_interview_df ORDER BY platform, usd_price')

,user_id,event_date,platform,usd_price
0,16752,2020-04-01,android,1.99
1,59801,2020-04-01,android,4.99
2,23598,2020-04-01,android,4.99
3,78345,2020-04-01,android,19.99
4,67895,2020-04-01,android,29.99
5,89234,2020-04-01,ios,1.99
6,43459,2020-04-01,ios,1.99
7,23456,2020-04-01,ios,2.99
8,43576,2020-04-01,ios,4.99
9,18809,2020-04-01,ios,9.99


### Q2

Using the `revenue_data_interview.csv` dataset, write a SQL query you'd use to generate this output:

``` 
|user_id
|install_source
|platform
|install_date
|all_time_revenue
|revenue_1_days_after_install
|revenue_7_days_after_install
|revenue_30_days_after_install
|revenue_60_days_after_install
```


This is a cumulative calculation of revenue.
For example **revenue_15_days_after_install = total revenue from the user 15 days after their install date.**

SOURCE: `https://dba.stackexchange.com/questions/107485/rolling-sum-count-average-over-date-interval`


In [12]:
file_name = "/content/drive/MyDrive/test/uken_interview_data_analyst/revenue_data_interview.csv"
df = pd.read_csv(file_name) ; df.head()

,install_date,event_time,user_id,platform,install_source,usd_price
0,2018-07-09,2018-07-10 08:31:00.150000,2344c8d2b7642d01792bb225e23f5491,android,facebook,4.99
1,2018-07-09,2018-07-10 08:36:14.701000,2344c8d2b7642d01792bb225e23f5491,android,facebook,1.99
2,2018-07-16,2018-07-17 21:17:27.746000,a1cb0872fa30cd5e4fd7b8eab0b92c83,android,facebook,4.99
3,2018-07-16,2018-07-18 18:42:28.613000,a1cb0872fa30cd5e4fd7b8eab0b92c83,android,facebook,4.99
4,2018-07-16,2018-07-18 18:49:39.003000,a1cb0872fa30cd5e4fd7b8eab0b92c83,android,facebook,4.99


In [24]:
# we'll use event_time as our unique id..
q0 = """
SELECT
    install_date,
    user_id,
    count(user_id) count
FROM df
GROUP BY install_date, user_id
ORDER BY count DESC
"""
pysqldf(q0)

,install_date,user_id,count
0,2019-03-02,403619b8da2877d9f7442e4cfd1a3a9d,113
1,2019-07-05,90f753c44a06e159af9c8697487c9b05,64
2,2019-02-13,45b0a591274aea746e7fe00938299345,54
3,2018-11-06,e4db8193db5529369bcdf0da070c690b,38
4,2020-02-11,ae7dd473862eea67081fc22050e41f5d,36
...,...,...,...
95,2020-06-28,6c766153d897113b8c5b2b8af43a77bd,1
96,2020-07-02,b3733ff228c68ebb8f54825713c42493,1
97,2020-07-12,1dea24d44cc25876fdbb5b3f07c589ef,1
98,2020-08-04,c04c0f49f2baa9cd0f5527d2a88bc23a,1


In [71]:
# looks like install date will always be the same, and we have to look at event_time
q0 = """
SELECT
    user_id,
    install_date,
    event_time,
    usd_price
FROM df 
WHERE user_id = '010bcfdfc42757e79888790a632ce492'
ORDER BY install_date
"""
pysqldf(q0)

,user_id,install_date,event_time,usd_price
0,010bcfdfc42757e79888790a632ce492,2019-07-14,2020-03-06 22:39:04.824000,1.99


In [75]:
q0 = """
SELECT
    t0.user_id,
    t0.install_date,
    event_time,
    sum(t1.usd_price) AS trans_total,
    count(*) AS trans_count
FROM   df t0
JOIN   df t1 USING (event_time)
WHERE  DATE(t1.event_time) BETWEEN DATE(t0.install_date) and DATE(t0.install_date, '+15 day')
GROUP  BY t0.user_id -- is PK!
ORDER  BY trans_count DESC
"""
pysqldf(q0)

,user_id,install_source,platform,install_date,event_time,trans_total,trans_count
0,ae7dd473862eea67081fc22050e41f5d,facebook,ios,2020-02-11,2020-02-26 23:17:25.380000,69.80,20
1,90f753c44a06e159af9c8697487c9b05,organic,ios,2019-07-05,2019-07-11 20:59:54.128000,49.81,19
2,9105edc8a9906d47af5ac9d8f386947d,organic,ios,2018-10-30,2018-11-13 04:13:16.347000,34.84,16
3,f37752acfe1d7db62ce648abd9460bb3,organic,ios,2020-03-27,2020-03-30 06:43:04.994000,23.91,9
4,c8ba2166c18f2577730c725f1c96a397,ironsource,ios,2018-12-30,2019-01-11 01:27:24.341000,27.92,8
...,...,...,...,...,...,...,...
74,eead0d61a660284d3b544378f800d068,facebook,android,2020-04-11,2020-04-11 20:09:33.574000,1.99,1
75,f501183889f4d90adb92e1b20183cd66,facebook,ios,2020-05-03,2020-05-03 18:15:58.819000,7.99,1
76,fb071282a868032ca6bb62063b31de59,google,ios,2019-04-20,2019-04-22 04:16:28.875000,4.99,1
77,fcf5879da9a831c969af60e8d6f3ae11,unity ads,ios,2019-02-16,2019-02-23 01:54:46.828000,4.99,1


In [83]:
# need 1, 7,30,60
q0 = """
SELECT
    t0.user_id,
    t0.install_source,
    t0.platform,
    t0.install_date,
    t1.all_time_revenue,
    t2.trans_total as revenue_1_days_after_install,
    t3.trans_total as revenue_7_days_after_install,
    t4.trans_total as revenue_30_days_after_install,
    t5.trans_total as revenue_60_days_after_install
FROM   df t0
LEFT JOIN (
    SELECT
        user_id,
        sum(usd_price) AS all_time_revenue
    FROM   df 
    GROUP  BY user_id
) AS t1 ON t0.user_id = t1.user_id
LEFT JOIN (
    SELECT
        t20.user_id,
        sum(t21.usd_price) AS trans_total
    FROM   df t20
    JOIN   df t21 USING (event_time)
    WHERE  DATE(t21.event_time) BETWEEN DATE(t20.install_date) and DATE(t20.install_date, '+1 day')
    GROUP  BY t20.user_id
) AS t2 ON t0.user_id = t2.user_id
LEFT JOIN (
    SELECT
        t30.user_id,
        sum(t31.usd_price) AS trans_total
    FROM   df t30
    JOIN   df t31 USING (event_time)
    WHERE  DATE(t31.event_time) BETWEEN DATE(t30.install_date) and DATE(t30.install_date, '+7 day')
    GROUP  BY t30.user_id
) AS t3 ON t0.user_id = t3.user_id
LEFT JOIN (
    SELECT
        t40.user_id,
        sum(t41.usd_price) AS trans_total
    FROM   df t40
    JOIN   df t41 USING (event_time)
    WHERE  DATE(t41.event_time) BETWEEN DATE(t40.install_date) and DATE(t40.install_date, '+30 day')
    GROUP  BY t40.user_id
) AS t4 ON t0.user_id = t4.user_id
LEFT JOIN (
    SELECT
        t50.user_id,
        sum(t51.usd_price) AS trans_total
    FROM   df t50
    JOIN   df t51 USING (event_time)
    WHERE  DATE(t51.event_time) BETWEEN DATE(t50.install_date) and DATE(t50.install_date, '+60 day')
    GROUP  BY t50.user_id
) AS t5 ON t0.user_id = t5.user_id
GROUP  BY t0.user_id -- is PK!
"""
pysqldf(q0)

,user_id,install_source,platform,install_date,all_time_revenue,revenue_1_days_after_install,revenue_7_days_after_install,revenue_30_days_after_install,revenue_60_days_after_install
0,010bcfdfc42757e79888790a632ce492,facebook,android,2019-07-14,1.99,NaN,NaN,NaN,NaN
1,054d14ab5aadf25111a4d69887d7ec8a,facebook,android,2020-04-18,1.99,NaN,NaN,1.99,1.99
2,05c0faf28078df557033f0de7c3ebd7b,organic,android,2018-09-21,1.99,1.99,1.99,1.99,1.99
3,094fabc4ed3460ffe6f7b9a589775c60,organic,ios,2019-05-29,1.99,NaN,1.99,1.99,1.99
4,11dd3513a4ec9d94c7ec605f165f6679,facebook,android,2018-08-21,3.98,3.98,3.98,3.98,3.98
...,...,...,...,...,...,...,...,...,...
95,f41ddf77a26f14f629043124af874a37,unattributed,ios,2020-04-06,5.97,NaN,NaN,NaN,NaN
96,f501183889f4d90adb92e1b20183cd66,facebook,ios,2020-05-03,19.97,7.99,7.99,17.98,19.97
97,fb071282a868032ca6bb62063b31de59,google,ios,2019-04-20,4.99,NaN,4.99,4.99,4.99
98,fcf5879da9a831c969af60e8d6f3ae11,unity ads,ios,2019-02-16,4.99,NaN,4.99,4.99,4.99


## Python

### Q1

Using the `retention_data_interview.csv` dataset, get the retention by install date, platform, and install source in the following output:

```
|install_date
|platform
|install_source
|day_1_retention
|day_7_retention
|day_30_retention
|day_360_retention
```

**Note: Retention = returns / installs**


In [84]:
file_name = "/content/drive/MyDrive/test/uken_interview_data_analyst/retention_data_interview.csv"
df = pd.read_csv(file_name) ; df.head()

,install_date,n,platform,install_source,installs,returns
0,2019-10-02,60,android,organic,4202,90
1,2019-10-02,15,android,organic,4202,226
2,2019-10-02,360,android,facebook,1877,4
3,2019-10-02,1,android,google,2010,569
4,2019-10-02,120,android,facebook,1877,18


In [110]:
# retention is unique for each install_date, platform_installsource
df[(df.install_source == 'organic') & (df.n==1) & (df.platform == 'android')]

,install_date,n,platform,install_source,installs,returns
25,2019-10-04,1,android,organic,4188,1025
207,2019-10-08,1,android,organic,4257,1119
241,2019-10-12,1,android,organic,4801,1306
261,2019-10-14,1,android,organic,4311,1073
279,2019-10-17,1,android,organic,4034,1132
324,2019-10-05,1,android,organic,4500,1111
344,2019-10-07,1,android,organic,4185,1094
453,2019-10-03,1,android,organic,4241,1034
528,2019-10-10,1,android,organic,4041,1032
537,2019-10-11,1,android,organic,4307,1005


In [112]:
df['retention'] = df.returns / df.installs ; df

,install_date,n,platform,install_source,installs,returns,retention
0,2019-10-02,60,android,organic,4202,90,0.021418
1,2019-10-02,15,android,organic,4202,226,0.053784
2,2019-10-02,360,android,facebook,1877,4,0.002131
3,2019-10-02,1,android,google,2010,569,0.283085
4,2019-10-02,120,android,facebook,1877,18,0.009590
...,...,...,...,...,...,...,...
1627,2019-10-22,7,ios,facebook,2252,326,0.144760
1628,2019-10-22,45,android,organic,4310,111,0.025754
1629,2019-10-22,150,ios,facebook,2252,24,0.010657
1630,2019-10-22,45,ios,google,1077,65,0.060353


In [123]:
df.query('n in [1,7,30,360]')

,install_date,n,platform,install_source,installs,returns,retention
2,2019-10-02,360,android,facebook,1877,4,0.002131
3,2019-10-02,1,android,google,2010,569,0.283085
6,2019-10-02,360,android,google,2010,7,0.003483
7,2019-10-02,1,ios,google,1583,610,0.385344
12,2019-10-03,7,android,google,2393,206,0.086084
...,...,...,...,...,...,...,...
1616,2019-10-21,30,ios,facebook,2367,141,0.059569
1617,2019-10-21,7,ios,organic,4123,487,0.118118
1622,2019-10-21,1,ios,google,1235,486,0.393522
1627,2019-10-22,7,ios,facebook,2252,326,0.144760


In [126]:
df2 = df.query('n in [1,7,30,360]').pivot_table('retention', ["install_date", "platform", "install_source"], 'n')
df2.columns = ['day_1_retention', 'day_7_retention','day_30_retention', 'day_360_retention']
df2.reset_index()

,install_date,platform,install_source,day_1_retention,day_7_retention,day_30_retention,day_360_retention
0,2019-10-02,android,facebook,0.346297,0.117208,0.038892,0.002131
1,2019-10-02,android,google,0.283085,0.086567,0.030348,0.003483
2,2019-10-02,android,organic,0.259638,0.080438,0.036411,0.004046
3,2019-10-02,ios,facebook,0.394173,0.171380,0.060840,0.003428
4,2019-10-02,ios,google,0.385344,0.142767,0.061276,0.012634
...,...,...,...,...,...,...,...
121,2019-10-22,android,google,0.322835,0.103487,0.041057,NaN
122,2019-10-22,android,organic,0.263573,0.085151,0.035267,NaN
123,2019-10-22,ios,facebook,0.395204,0.144760,0.053730,NaN
124,2019-10-22,ios,google,0.402043,0.174559,0.086351,NaN


### Q2

Write an efficient algorithm that searches for a value in an m x n matrix. This matrix has the following properties:

Integers in each row are sorted in ascending from left to right.

```
[
    [1, 3, 5, 11, 12],
    [12, 33, 44, 55, 60],
    [2, 6, 8, 9, 10, 29],
    [11, 22, 33, 45, 62],
    [3, 5, 8, 23, 24]
]
```

Given target = `5` return `True`. 

Given target = `13` return `False`.

You can use the test cases below to check your work.

SOURCE: https://stackoverflow.com/questions/14902367/divide-and-conquer-indexsearch

In [137]:
from typing import List

def search_matrix_int(matrix: List[List[int]], target: int) -> bool: 
    """
    1. iter through matrix: n
    2. run each row with search function: O(nLogn)
    
    Total runtime: n* nLogn = n^2 logn
    """

    def search(row):     
        if len(row) == 1:
            return row[0] == target
        else:
            m = len(row) //2
            return search(row[m:]) or search(row[:m])

    for row in matrix:
        if search(row):
            return True

    return False


In [138]:
##Test case 1
target_1 = 12 # True
target_2 = 22 # False

matrix_1 = [[1, 2, 3, 4, 5], [12, 13, 14, 15, 16], [21, 23, 24, 25, 26]]

In [139]:
search_matrix_int(matrix_1,target_1),search_matrix_int(matrix_1,target_2) 

(True, False)

In [18]:
##Test case 2
target_3 = 8 # False
target_4 = 24 # True
target_5 = 0 # True

matrix_2 = [[1, 2, 3, 4, 5], [12, 13, 14, 15, 16], [21, 23, 24, 25, 26], [66, 77, 88, 99, 100], [3, 4, 6, 7, 9], [0, 10, 20, 30, 40]]

In [140]:
search_matrix_int(matrix_2,target_3),search_matrix_int(matrix_2,target_4),search_matrix_int(matrix_2,target_5)

(False, True, True)

## General Business / Stats

An A/B test on a new feature / campaign is going to be evaluated on 20 different independent metrics. 

`1 out of the 20` shows a statistical significant change. 

Interpret the results. Please list out any assumptions or caveats to your answer. 

In [18]:
# i give up